# Desafio técnico *Linear Engenharia TI*
<hr>
<font size=2>Autor: <a href='https://github.com/baiochi'>João Baiochi  

In [1]:
# TODO

# - FINALIZAR PERGUNTAS/QUESTOES
# - Rever dataframe com merge em tarifas e consumo

# Workflow  

1. [Descrição do desafio](#item1)  
2. [Bibliotecas e APIs  ](#item2)  
> Descrição das bibliotecas utilizadas na exploração e análise dos dados.
3. [Leitura dos dados](#item3)  
> Carregando o arquivo e viualizando a estrutura dos dados.
4. [Descrição das *features*](#item4)  
> Entender qual o significado das features e labels para melhor contextualizar o problema,  
Definir as perguntas.
5. [Tratamento dos dados](#item5)  
> Organização dos dataframes,  
Atribuição dos tipos corretos,  
Tratamento de dados faltantes.  
6. [Análise dos dados](#item6)   
> Distribuição das features,
Responder as perguntas geradas.
7. [Apresentação com *Dashboards* ](#item7)  
> Cloud deployment com o *Streamlit*.  
8. [Possíveis aplicações de ML](#item8)  
> Algumas abordagens de Machine Learning que possam ser utilizadas.  

# 1. <span id='item1'>Descrição do desafio

# 2. <span id='item2'>Bibliotecas e APIs

In [2]:
# Leitura de arquivos .json
import json
# Operações matemáticas
import numpy as np
# Operações com regex
import re
# Leitura e manipulação de dados
import pandas as pd
# Visualização interativa dos dados
import plotly.express as px
# Customizações para visualização os dados
from utils.urban_layout import CATEGORICAL_GROUPS, LAYOUT_SPECS, SHADES

In [3]:
# Remove o limite de colunas a serem mostradas
pd.set_option('display.max_columns', None)

# 3. <span id='item3'>Leitura dos dados

Para ler os dados, podemos tanto usar a bilioteca `json` ou `pandas`, ambos fornecem meios diferentes para fazer a extração e organização nos dataframes.

In [4]:
# Carregando com json
with open('data/consumo.json', 'r') as file:
  json_data = json.load(file)
# Carregando com pandas
df = pd.read_json('data/consumo.json')

# Mantendo uma cópia dos arquivos orginais
raw_df = df.copy()
raw_json_data = json_data.copy()

`pandas` fornece uma rápido sumário dos dados: quais os tipos, se há dados faltantes(null), e quantidade de obervações.  

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162 entries, 0 to 161
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   _id                   162 non-null    object
 1   estruturaConsumo      162 non-null    object
 2   unique                162 non-null    object
 3   cliente               162 non-null    object
 4   classe                162 non-null    object
 5   ligacao               162 non-null    object
 6   referencia            162 non-null    object
 7   data_vencimento       162 non-null    object
 8   data_emissao          162 non-null    object
 9   data_apresentacao     162 non-null    object
 10  data_proxima_leitura  162 non-null    object
 11  cnpj                  162 non-null    object
 12  uc                    162 non-null    object
 13  refaturamento         162 non-null    bool  
 14  tarifas               162 non-null    object
 15  createdAt             162 non-null    ob

Também podemos visualizar as 5 primeiras linhas para ver como os dados foram ineridos. 

In [6]:
df.head()

,_id,estruturaConsumo,unique,cliente,classe,ligacao,referencia,data_vencimento,data_emissao,data_apresentacao,data_proxima_leitura,cnpj,uc,refaturamento,tarifas,createdAt,updatedAt,__v
0,{'$oid': '62220ff138b5b03262a806d4'},"{'saldoPonta': '0', 'saldoForaPonta': '0', 'ex...",b45dc7a8-2aa8-4693-a7c2-2260c4f0b642,SUPERMERCADO PAREDAO LTDA,COMERCIAL / OUTROS SERVIÇOS E OUTRAS ATIVIDADES,TRIFASICO,2022-02,2022-04-11,2022-02-26,2022-03-04,2022-03-29,28.847.082/0001-23,6/732146-6,False,"[{'cci': '0601', 'descricao': 'CONSUMO EM KWH'...",{'$date': '2022-03-04T13:11:13.417Z'},{'$date': '2022-03-04T13:11:13.417Z'},0
1,{'$oid': '62220ff138b5b03262a806da'},"{'saldoPonta': '0', 'saldoForaPonta': '0', 'ex...",1bcac16e-289a-45c4-8509-f98ad5b66bcb,TELEFONICA BRASIL SA,COMERCIAL / SERVIÇOS DE COMUNICAÇÕES E TELECOM...,TRIFASICO,2022-02,2022-03-15,2022-02-26,2022-03-08,2022-03-29,02.558.157/0001-62,6/2669086-7,False,"[{'cci': '0601', 'descricao': 'CONSUMO EM KWH'...",{'$date': '2022-03-04T13:11:13.565Z'},{'$date': '2022-03-04T13:11:13.565Z'},0
2,{'$oid': '62220ff238b5b03262a806e0'},"{'saldoPonta': '0', 'saldoForaPonta': '0', 'ex...",37399592-abd1-4de3-bb28-1b3753caf972,TELEFONICA BRASIL SA,COMERCIAL / SERVIÇOS DE COMUNICAÇÕES E TELECOM...,BIFASICO,2022-02,2022-03-28,2022-02-26,2022-03-11,2022-03-29,02.558.157/0001-62,6/2668744-2,False,"[{'cci': '0601', 'descricao': 'CONSUMO EM KWH'...",{'$date': '2022-03-04T13:11:14.121Z'},{'$date': '2022-03-04T13:11:14.121Z'},0
3,{'$oid': '62220ff338b5b03262a806ee'},"{'saldoPonta': '16769', 'saldoForaPonta': '180...",c0185cf7-f793-4c23-a593-b1ec1d41219a,COMERCIAL IOMAR DE ALIMENTOS LTDA EPP,COMERCIAL / COMERCIAL,TRIFASICO,2022-02,2022-04-13,2022-03-02,2022-03-04,2022-03-31,01.436.106/0001-03,6/336632-5,False,"[{'cci': '0601', 'descricao': 'CONSUMO EM KWH ...",{'$date': '2022-03-04T13:11:15.541Z'},{'$date': '2022-03-04T13:11:15.541Z'},0
4,{'$oid': '62220ff538b5b03262a80702'},"{'saldoPonta': '222726', 'saldoForaPonta': '24...",161ce259-5c70-4818-b2c5-2eee335e5067,CONSORCIO GERADOR FAZENDA MODELO,COMERCIAL / OUTROS SERVIÇOS E OUTRAS ATIVIDADES,TRIFASICO,2022-03,2022-03-16,2022-03-03,2022-03-09,2022-03-30,33.372.528/0001-40,6/2731936-7,False,"[{'cci': '0602', 'descricao': 'DEMANDA POTÊNCI...",{'$date': '2022-03-04T13:11:17.866Z'},{'$date': '2022-03-04T13:11:17.866Z'},0


De início observa-se que várias colunas não estão num formato adequado, por exemplo as referentes as datas.  
Algumas colunas também possuem dataframes aninhados.

In [7]:
# Verificando quais colunas pertencem ao tipo dicionáro ou lista
[column for column in df.columns if isinstance(df[column][0], (dict, list))]

['_id', 'estruturaConsumo', 'tarifas', 'createdAt', 'updatedAt']

As colunas *_id*, *createdAt* e *updatedAt* podem ser normalizadas facilmente uma vez que possuem apenas 1 campo "chave-valor". Já as colunas *estruturaConsumo* e *tarifas* precisam ser analisadas com mais atenção, por conterem uma maior quantidade de informação e talvez mais dataframes aninhados.

## Analisando a coluna *estruturaConsumo*

In [8]:
# Achatando os dados dentro da coluna estruturaConsumo
estruturaConsumo_df = pd.json_normalize(df['estruturaConsumo'])
print(estruturaConsumo_df.dtypes)
estruturaConsumo_df.head(1)

saldoPonta         object
saldoForaPonta     object
expiraForaPonta    object
expiraPonta        object
saldoAcumulado     object
expiraAcumulado    object
leituraAnterior    object
leituraAtual       object
leituras           object
dtype: object


,saldoPonta,saldoForaPonta,expiraForaPonta,expiraPonta,saldoAcumulado,expiraAcumulado,leituraAnterior,leituraAtual,leituras
0,0,0,0,0,0,0,2022-01-28,2022-02-25,"[{'un': 'KWH', 'posto': 'PONTA', 'medido': '24..."


In [9]:
estruturaConsumo_df['leituras'][0]

[{'un': 'KWH',
  'posto': 'PONTA',
  'medido': '24363',
  'faturado': '24363',
  '_id': {'$oid': '62220ff138b5b03262a806d9'}}]

O campo *estruturaConsumo* possuí algumas colunas com valores numéricos, data e uma com lista contendo outros dataframes. Esta é mais uma coluna que iremos normalizar, lembrando que uma observação do campo *estruturaConsumo* pode conter mais de um tipo de leitura.  

In [10]:
# Selecionando o dado a ser plotado
# Retorna um pd.Series com o tamanho de cada lista por linha
plot_data = estruturaConsumo_df['leituras'].apply(lambda x : len(x))

#
px.histogram(
	data_frame=plot_data,	    					# Dado a ser plotado
	title='Distribuição da quantidade de leituras',	# Título do gráfico
	width=800, opacity=0.8							# Tamanho e alpha da cor
).update_traces(											
	marker_color=SHADES['CYAN'][3], 				# Customização do layput da barra	
	marker_line_color=SHADES['CYAN'][6], 
	marker_line_width=1.5,
).update_layout(
	showlegend=False, 								# Esconde a legenda
	xaxis_title='Número de tarifas',				# Renomeia título do eixo x 
	yaxis_title='Quantidade',						# Renomeia título do eixo y 
	**LAYOUT_SPECS									# Customizações do layout
)

Para analisar os campos desta coluna *leituras*, vamos normalizar os dados separadamente.

In [11]:
leituras_df = pd.json_normalize(
	df['estruturaConsumo'], 
	record_path=['leituras']
)
leituras_df.head()

,un,posto,medido,faturado,_id.$oid
0,KWH,PONTA,24363,24363,62220ff138b5b03262a806d9
1,KWH,PONTA,1005,1005,62220ff138b5b03262a806df
2,KWH,PONTA,1000,1000,62220ff238b5b03262a806e6
3,KWH,PONTA,1856.97,1856.97,62220ff338b5b03262a806f8
4,INJ,PONTA,0,1856.97,62220ff338b5b03262a806f9


In [12]:
for coluna in ['un', 'posto']:
	print(f'Labels da coluna "{coluna}": {leituras_df[coluna].unique()}')

Labels da coluna "un": ['KWH' 'INJ' 'KW' 'ERE' 'DRE' 'KVA' 'ULTP']
Labels da coluna "posto": ['PONTA' 'FPONTA']


Por enquanto concluímos quais *features* e *labels* devemos pesquisar para entender melhor o contexto.

## Analisando a coluna *tarifas*

In [13]:
df['tarifas'][:5]

0    [{'cci': '0601', 'descricao': 'CONSUMO EM KWH'...
1    [{'cci': '0601', 'descricao': 'CONSUMO EM KWH'...
2    [{'cci': '0601', 'descricao': 'CONSUMO EM KWH'...
3    [{'cci': '0601', 'descricao': 'CONSUMO EM KWH ...
4    [{'cci': '0602', 'descricao': 'DEMANDA POTÊNCI...
Name: tarifas, dtype: object

Esta coluna já se difere por ser uma lista contendo mais de um dataframe. Vamos aplicar o gráfico usado anteriormente para visualizar a distribuição do número de tarifas.

In [14]:
# Selecionando o dado a ser plotado
# Retorna um pd.Series com o tamanho de cada lista por linha
plot_data = df['tarifas'].apply(lambda x : len(x))
# Plotando com plotly express
px.histogram(
	data_frame=plot_data,	    					# Dado a ser plotado
	title='Distribuição da quantidade de tarifas',	# Título do gráfico
	width=800, opacity=0.8							# Tamanho e alpha da cor
).update_traces(											
	marker_color=SHADES['YELLOW'][3], 				# Customização do layout da barra	
	marker_line_color=SHADES['YELLOW'][6], 
	marker_line_width=1.5,
).update_layout(
	showlegend=False, 								# Esconde a legenda
	xaxis_title='Número de tarifas',				# Renomeia título do eixo x 
	yaxis_title='Quantidade',						# Renomeia título do eixo y 
	**LAYOUT_SPECS									# Customizações do layout
)

A distribuição das tarifas é bem diferente das leituras, portanto podemos questionar qual a relação entre elas, ou seja, se o número de tarifas aplicadas tem correlação entre a quantidade de itens medidos na *estruturaConsumo*.

In [15]:
# Calculando a diferença entre quantidade de tarifas e quantidade
# de medições feitas na estruturaConsumo de uma fatura
dist_tarifas = df['tarifas'].apply(lambda x : len(x))
dist_leituras = estruturaConsumo_df['leituras'].apply(lambda x : len(x))
plot_data = dist_tarifas - dist_leituras
p_corr = dist_tarifas.corr(dist_leituras)

px.line(
	data_frame = plot_data,							# Dado a ser plotado
	title=f'Diferença entre # de tarifas\
 e medições na estruturaConsumo<br>\
	<sup>p = {p_corr:.4f}</sup>', 					# Título do gráfico
	color_discrete_sequence=[SHADES['CYAN'][4]], 	# Cor da linha
	width=800, 									 	# Tamanho 
).add_shape(
	type='line', x0=0, y0=0, x1=160, y1=0, 			 # Desenha uma linha horizontal
	line_color=SHADES['GRAY'][4]
).update_layout(
	showlegend=False, 							 	 # Esconde a legenda
	xaxis_title='Índice da Fatura',				     # Renomeia título do eixo x 
	yaxis_title='Diferença',						 # Renomeia título do eixo y 
	**LAYOUT_SPECS									 # Customizações do layout
)

Essa diferença possue uma leve correlação positiva, mas em alguns casos há mais quantidades de medições do que tarifas aplicadas na fatura.   
  
Agora vamos analisar quais os campos da coluna *tarifas*. Neste caso iremos utilizar a normalização através dos dados carregados pela biblioteca `json`

In [16]:
# Achatando os dados da coluna tarifas
tarifas_df = pd.json_normalize(
	json_data,					# Dado carregado com json
	record_path=['tarifas']	# Caminho com os dados para normalizar
)
print(tarifas_df.dtypes)
tarifas_df.head(3)

cci               object
descricao         object
quantidade       float64
semTributos      float64
comTributos      float64
valorTotal       float64
baseIcms         float64
aliqIcms         float64
icms             float64
basePisCofins    float64
pis              float64
cofins           float64
_id.$oid          object
dtype: object


,cci,descricao,quantidade,semTributos,comTributos,valorTotal,baseIcms,aliqIcms,icms,basePisCofins,pis,cofins,_id.$oid
0,0601,CONSUMO EM KWH,24363.0,0.6835,0.85548,20842.28,20842.28,17.0,3543.18,17299.10,115.40,531.58,62220ff138b5b03262a806d5
1,0601,ENERGIA ATV INJETADA OUC 02/2022 MPT,12429.0,0.6835,0.76897,-9557.65,-4308.07,17.0,-732.37,-8825.28,-58.87,-271.19,62220ff138b5b03262a806d6
2,0601,ADIC. B. VERMELHA,0.0,0.0000,0.00000,2121.05,2121.05,17.0,360.58,1760.47,11.74,54.09,62220ff138b5b03262a806d7


Desta vez o formato dos arquivos foram carregados corretamente. Por último vamos analisar quais são as categorias para as colunas *cci* e *descricao*.  

In [17]:
print(f"Categorias para cci: {tarifas_df['cci'].unique()}")

Categorias para cci: ['0601' '0807' '0904' '0602' '0999' '0899' '0603' '0805' '0804' '0610'
 '0906' '0806']


In [18]:
print(f"Categorias para descricao: {tarifas_df['descricao'].unique()}")

Categorias para descricao: ['CONSUMO EM KWH' 'ENERGIA ATV INJETADA OUC 02/2022 MPT'
 'ADIC. B. VERMELHA' 'CONTRIB DE ILUM PUB'
 'COMPENS. P/INDICADOR-FIC ANUAL 12/2021' 'CONSUMO EM KWH - PONTA'
 'ENERGIA ATV INJETADA OUC 2/2022 MPT' 'CONSUMO EM KWH - FORA PONTA'
 'ENERGIA REATIVA EXCED EM KWH - FPONTA'
 'DEMANDA DE POTÊNCIA MEDIDA - FORA PONTA'
 'DEMANDA POTÊNCIA NÃO CONSUMIDA - F PONTA'
 'DEV. DE ICMS JUDICIAL ( - ): ISENÇÃO ICMS S/ TUSD'
 'ENERGIA REATIVA EXCED EM KWH - PONTA' 'DIF. CUSTO DISP. RES. 482.'
 'ENERGIA ATV INJETADA OUC 03/2022 MPT' 'CREDITO ANTERIOR 02/2022'
 'DEV. DE ICMS JUDICIAL ( - ) : ISENÇÃO ICMS S/ TUSD'
 'DIF.CREDITO A DEVOLVER 03/2022' 'ENERGIA ATV INJETADA MUC 12/2021 MPT'
 'ENERGIA ATV INJETADA OUC 3/2022 MPT' 'ENERGIA REATIVA EXCED EM KWH'
 'CUSTO DE DISPONIBILIDADE' 'ENERGIA ATV INJETADA MUC 1/2022 MPT'
 'DEMANDA DE POTÊNCIA MEDIDA - PONTA'
 'DEMANDA POTÊNCIA NÃO CONSUMIDA - PONTA'
 'DEMANDA POTÊNCIA REATIVA EXCEDENTE PONTA'
 'DEMANDA POTÊNCIA REATIVA EXCED 

# 4. <span id='item4'> Descrição das *features*

Após esta breve visualização de como os dados estão estruturados, vamos entender melhor o significado das *features* e *labels* encontradas, para conseguir elaborar perguntas que irão definir o rumo da análise.  
  
As informacões a seguir foram retiradas através de consultas no site da [Energisa](https://www.energisa.com.br/) e da [Agência Nacional de Energia Elétrica (ANEEL)](https://www.gov.br/aneel/pt-br/centrais-de-conteudos/glossario).

### Termos gerais

**Consumo**: Quantidade de energia consumida em um mês, em KWH.  
**Demanda**: Potência ativa a ser obrigatória e continuamente disponibilizada pela distribuidora, em KW.  

**Postos tarifários**  
Períodos de tempo do dia definidos pela distribuidora.  
- Ponta: 3 horas consecutivas, onde o consumo de energia pela rede é maior.
A tarifa nesse período é mais cara.
- Fora de Ponta: demais horas do dia, a tarifa é mais barata.  

**TE**: Tarifa de Energia (KWh)  
**TUSD**: Tarifa de Uso do Sistema de Distribuição  

**Energia consumida**: parcelas $TE$ e $TUSD_{energia}$ em $R$/KWh$  
**Demanda contratada**: apenas a parcela $TUSD_{demanda}$ em $R$/KW$  
  
<font color='#1696D2'>*Tarifa Azul*
- $TUSD_{energia}$ única
- $TUSD_{demanda}$ Ponta/FPonta
- $TE$ Ponta/FPonta</font>  

<font color='#55B748'>*Tarifa Verde*
- $TUSD_{energia}$ Ponta/FPonta
- $TUSD_{demanda}$ única
- $TE$ Ponta/FPonta</font>  

Em resumo, a tarifa azul é mais atraente para locais que tem a necessidade de consumir mais energia no período de Ponta, pois dependendo da consecionária, a tarifa $TUSD_{energia}$ única costuma ser igual a $TUSD_{energia}$ FPonta. Por outro lado, a tarifa verde é interessante para locais que conseguem economizar energia no período de Ponta ou que necessitem de uma demanda maior.

  ### Signifcado das labels do campo *leituras*:  

- KWH: Consumo, $TUSD_{energia}$
- KW: Demanda de potência medida, $TUSD_{demanda}$
- KVA: Energia reativa, [cobrada caso fator de potência < 0.92](http://www2.aneel.gov.br/aplicacoes/audiencia/arquivo/2012/065/resultado/ren2013569.pdf)  
- ERE: Energia reativa excedente  
- DRE: Demanda reativa excedente  
- INJ: Quantidade de energia elétrica injetada nas redes do sistema  
- ULTP: Ultrapasagem da demanda contratada

### Visualizando as labels mais comuns da coluna *descricao* em *tarifas*

In [19]:
# Seleciona apenas descrições com os termos mais comuns
query = tarifas_df[tarifas_df['descricao'].str.contains('CONSUMO|ENERGIA|DEMANDA|MULTA')]
# Excluí os valores numéricos (datas)
query_results = query['descricao'].apply(lambda x : ' '.join([char for char in x.split() if char.isalpha()]))

In [20]:
# Selecionando o dado a ser plotado
plot_data = pd.DataFrame({
	'descricao' : query_results,
	'classe' : query_results.apply(lambda x : x.split()[0])
}).sort_values(by='descricao', ascending=False)
# Plotando com plotly express
px.histogram(
	data_frame=plot_data,	    					# Dado a ser plotado
	y='descricao',									# Muda a orientacão para a vertical
	color='classe',									# Separa em classes
	color_discrete_sequence=CATEGORICAL_GROUPS[4],	# Define as cores dos grupos
	title='Descrições das tarifas mais comuns',		# Título do gráfico
	width=800, opacity=0.8							# Tamanho e alpha da cor
).update_traces(												
	marker_line_color=SHADES['GRAY'][6], 			# Customização do layout da barra	
	marker_line_width=1.5,
).update_layout(
	legend_traceorder='reversed',					# Corrige a legenda em orem alfabética
	xaxis_title='Quantidade',						# Renomeia título do eixo x 
	yaxis_title='Descrição',						# Renomeia título do eixo y 
	**LAYOUT_SPECS									# Customizações do layout
)

Algumas descrições pertencem à categoria do posto **Ponta/FPonta**:  
- Consumo;  
- Demanda medida;
- Demanda não consumida; 
- Demanda ultrapassada;
- Demanda reativa excedente. 

Outras são inependentes dessa classificação:
- Energia ativa injetada;
- Energia reativa excedente.

### Questões a serem analisadas:
- Consumo (KWh) é maior na PONTA ou FPONTA?  
- Demanda contratada (KW) está sendo utilizada acima ou abaixo?  
- Relação entre a diferença do valor faturado e o valor medido. Se o valor medido for maior, qual o motivo do faturado ser menor? E no caso contrário? Existe algum comportamento comum para cada caso?
- Como se comporta a fatura onde há alto valor medido de energia injetada(INJ)? Essa energia é proveniente de geração própria?  


# 5. <span id='item5'>Tratamento dos dados

Antes de iniciar a análise exploratória, vamos aplicar alguns ajustes.

## Features com tipo *dict*

Iniciando pelo primeiro nível do arquivo *json*, será normalizada as colunas contendo valores do tipo *dict*, exceto as que contém outros valores aninhados (esturturaConsumo e tarifas).

In [21]:
# Achatando os dados
df['_id'] = pd.json_normalize(df['_id'])
df['createdAt'] = pd.json_normalize(df['createdAt'])
df['updatedAt'] = pd.json_normalize(df['updatedAt'])

## Features do tipo *string*

Agora conseguimos visualizar melhor as colunas com tipo *string*

In [22]:
string_columns = [
	'_id', 'unique', 'cliente', 
	'classe', 'ligacao', 'cnpj', 'uc',
]
print('Quantidade de valores únicos por feature')
(df[string_columns]
.apply(lambda x : x.nunique()))

Quantidade de valores únicos por feature


_id        162
unique     162
cliente     18
classe      10
ligacao      2
cnpj        17
uc          48
dtype: int64

Temos valores únicos tanto para `_id` quanto parece `unique`, entretando  a feature `_id` se assemelha mais com um identificador relacionado a coleta dos dados, enquanto `unique` é a chave da fatura.  
Vamos observar a relação desta chave com as demais features.

In [23]:
# Selecionando o dado a ser plotado
plot_columns = ['classe', 'ligacao', 'cnpj', 'uc']
plot_data = (
	df.groupby(by='cliente')				# Agrupando por cliente
	[plot_columns]							# Selecionando as colunas de interesse
	.count()								# Contando os valores
	.reset_index()							# Transformando para o formato "tidy"
	.melt(
		id_vars=['cliente'],
		var_name='Feature', 
		value_name='Valor')
	).sort_values(by='cliente', ascending=False)	# Ordenando clientes por ordem alfabética
# Plotando com plotly express
px.histogram(
	data_frame=plot_data,	    					# Dado a ser plotado
	x='Valor',
	y='cliente',									# Muda a orientacão para a vertical
	color='Feature',
	color_discrete_sequence=CATEGORICAL_GROUPS[len(plot_columns)],
	barmode='group',
	title='Distribuição das features por cliente',	# Título do gráfico
	width=800, opacity=0.8							# Tamanho e alpha da cor
).update_traces(									
	marker_line_color=SHADES['GRAY'][6], 			# Customização do layout da barra
	marker_line_width=1.5,
).update_layout(
	xaxis_title='Quantidade',						# Renomeia título do eixo x 
	yaxis_title='Cliente',							# Renomeia título do eixo y 
	**LAYOUT_SPECS									# Customizações do layout
)

Podemos observar que o cliente 'TELEFONICA BRASIL SA' está escrito de duas maneiras, isso explica porque existe 18 registros únicos de `clientes` e 17 de `cnpj`.  

In [24]:
# Renomeando o valor alternativo para o cliente
df['cliente'] = df['cliente'].replace({'TELEFONICA BRASIL S A': 'TELEFONICA BRASIL SA'})

A coluna `__v` possuí apenas 1 valor, portanto iremos removê-la do dataframe

In [25]:
# Contagem dos valores da coluna
print(df['__v'].value_counts())
# Aplicando o drop da coluna
df.drop(columns='__v', inplace=True)

0    162
Name: __v, dtype: int64


## Features do tipo *datetime*

Atualizando o *dtype* das colunas com data para *datetime*

In [26]:
# Selecionando as colunas com data
colunas_date = [
	'referencia', 'data_vencimento', 'data_emissao', 
	'data_apresentacao', 'data_proxima_leitura',
	'createdAt', 'updatedAt',
	]
# Aplicando o cast para datetime
for coluna in colunas_date:
	df[coluna] =  pd.to_datetime(df[coluna])

## `estruturaConsumo`

Para a coluna `estruturaConsumo` vamos realizar a normalização feita anteriormente, dessa vez adicionando como metadados os valores do nível raiz, dessa forma conseguimos ter uma chave para cada linha resultante do achatamento na coluna `leituras`.

In [27]:
# Adiciona os ids unique como nova chave para cada linha de estruturaConsumo
for index, id_value in enumerate(df['unique']):
	df['estruturaConsumo'].iloc[index]['unique'] = id_value

# Seleciona as colunas e metadados (todas exceto leituras)
meta_columns = list(pd.json_normalize(df['estruturaConsumo']).drop(columns='leituras'))

# Normaliza os dados
consumo_df = pd.json_normalize(
	df['estruturaConsumo'], 
	record_path=['leituras'],
	meta = meta_columns
)
print(f'Tipo dos dados: {set(consumo_df.dtypes)}')
consumo_df.head(3)

Tipo dos dados: {dtype('O')}


,un,posto,medido,faturado,_id.$oid,saldoPonta,saldoForaPonta,expiraForaPonta,expiraPonta,saldoAcumulado,expiraAcumulado,leituraAnterior,leituraAtual,unique
0,KWH,PONTA,24363,24363,62220ff138b5b03262a806d9,0,0,0,0,0,0,2022-01-28,2022-02-25,b45dc7a8-2aa8-4693-a7c2-2260c4f0b642
1,KWH,PONTA,1005,1005,62220ff138b5b03262a806df,0,0,0,0,0,0,2022-01-28,2022-02-25,1bcac16e-289a-45c4-8509-f98ad5b66bcb
2,KWH,PONTA,1000,1000,62220ff238b5b03262a806e6,0,0,0,0,0,0,2022-01-28,2022-02-25,37399592-abd1-4de3-bb28-1b3753caf972


Os daods foram normalizados, porém todas colunas estão do tipo *object*. Quando há claramente valores numéricos em uma coluna e o mesmo é convertido para *object*, pode significar que existe algum valor não numérico, como uma string vazia. Vamos fazer essa verificação.

In [28]:
consumo_df.apply(lambda x: x == '').sum()

un                  0
posto               0
medido              0
faturado            0
_id.$oid            0
saldoPonta         56
saldoForaPonta     56
expiraForaPonta    56
expiraPonta        56
saldoAcumulado     56
expiraAcumulado    56
leituraAnterior     8
leituraAtual        8
unique              0
dtype: int64

De fato, as colunas referentes aos saldos e datas possuem alguns valores de strings vazias.
Será necessário preencher esses campos com valores nulos(NaN) antes de fazer a atualização dos tipos de dados.

In [29]:
# Preenche strings vazias com valores nulos
consumo_df = consumo_df.replace(r'^\s*$', np.nan, regex=True)

# Selecionando as colunas para ser modificadas
int_columns = [
	'saldoPonta', 'saldoForaPonta', 
	'expiraForaPonta', 'expiraPonta', 'saldoAcumulado', 
	'expiraAcumulado']
float_columns = ['medido', 'faturado']
date_columns = ['leituraAnterior', 'leituraAtual']

# Aplicando os casts
consumo_df[int_columns]       = consumo_df[int_columns].astype('Int64')
consumo_df[float_columns]     = consumo_df[float_columns].astype('float64')
consumo_df['leituraAnterior'] = pd.to_datetime(consumo_df['leituraAnterior'])
consumo_df['leituraAtual']    = pd.to_datetime(consumo_df['leituraAtual'])

# Verificando o sumário do dataframe
consumo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 362 entries, 0 to 361
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   un               362 non-null    object        
 1   posto            362 non-null    object        
 2   medido           362 non-null    float64       
 3   faturado         362 non-null    float64       
 4   _id.$oid         362 non-null    object        
 5   saldoPonta       306 non-null    Int64         
 6   saldoForaPonta   306 non-null    Int64         
 7   expiraForaPonta  306 non-null    Int64         
 8   expiraPonta      306 non-null    Int64         
 9   saldoAcumulado   306 non-null    Int64         
 10  expiraAcumulado  306 non-null    Int64         
 11  leituraAnterior  354 non-null    datetime64[ns]
 12  leituraAtual     354 non-null    datetime64[ns]
 13  unique           362 non-null    object        
dtypes: Int64(6), datetime64[ns](2), float64(2)

Atualizando a coluna `estruturaConsumo` com os valores tratados.

In [30]:
# Convertendo o dataframe de volta ao formato dict
df['estruturaConsumo'] = consumo_df.apply(lambda x : x.to_dict(), axis=1)

## `tarifas`

Para melhor acessar as informações, vamos carregar novamente a coluna tarifas, dessa vez associando como metadado o id `unique` da fatura.

In [31]:
tarifas_df = pd.json_normalize(
	json_data,					# Dado carregado com json
	record_path=['tarifas'],	# Caminho para achatar
	meta=['unique']				# Coluna de metadado
)
tarifas_df.head(3)

,cci,descricao,quantidade,semTributos,comTributos,valorTotal,baseIcms,aliqIcms,icms,basePisCofins,pis,cofins,_id.$oid,unique
0,0601,CONSUMO EM KWH,24363.0,0.6835,0.85548,20842.28,20842.28,17.0,3543.18,17299.10,115.40,531.58,62220ff138b5b03262a806d5,b45dc7a8-2aa8-4693-a7c2-2260c4f0b642
1,0601,ENERGIA ATV INJETADA OUC 02/2022 MPT,12429.0,0.6835,0.76897,-9557.65,-4308.07,17.0,-732.37,-8825.28,-58.87,-271.19,62220ff138b5b03262a806d6,b45dc7a8-2aa8-4693-a7c2-2260c4f0b642
2,0601,ADIC. B. VERMELHA,0.0,0.0000,0.00000,2121.05,2121.05,17.0,360.58,1760.47,11.74,54.09,62220ff138b5b03262a806d7,b45dc7a8-2aa8-4693-a7c2-2260c4f0b642


Agora vamos adicionar uma nova coluna referente ao tipo da descrição, e uma ao posto.

In [32]:
tarifas_df['class_desc'] = tarifas_df['descricao'].apply(lambda x: x.split()[0])

In [33]:
# Filtrando a descrição com regex
tarifas_df['tarifa_posto'] = tarifas_df['descricao'].apply(
	lambda x: re.findall('PONTA|FORA PONTA|F PONTA|FPONTA', x))
# Eliminando o formato de lista
tarifas_df['tarifa_posto'] = tarifas_df['tarifa_posto'].apply(lambda x : x[0] if len(x)>0 else 'NA')
# Normalizando as labels
tarifas_df['tarifa_posto'] = tarifas_df['tarifa_posto'].replace({
	'F PONTA' : 'FPONTA',
	'FORA PONTA' : 'FPONTA',
})

In [34]:
tarifas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 777 entries, 0 to 776
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   cci            777 non-null    object 
 1   descricao      777 non-null    object 
 2   quantidade     777 non-null    float64
 3   semTributos    777 non-null    float64
 4   comTributos    777 non-null    float64
 5   valorTotal     777 non-null    float64
 6   baseIcms       777 non-null    float64
 7   aliqIcms       777 non-null    float64
 8   icms           777 non-null    float64
 9   basePisCofins  777 non-null    float64
 10  pis            777 non-null    float64
 11  cofins         777 non-null    float64
 12  _id.$oid       777 non-null    object 
 13  unique         777 non-null    object 
 14  class_desc     777 non-null    object 
 15  tarifa_posto   777 non-null    object 
dtypes: float64(10), object(6)
memory usage: 97.2+ KB


Atualizando a coluna `tarifas` com os valores tratados.

In [35]:
# Retornando ao formato de dicionários armazenados em lista
tarifas_list = (
	tarifas_df
	.groupby('unique')
	.apply(lambda x : x[tarifas_df.drop(columns='unique').columns].to_dict('records'))
	.reset_index()
	.rename(columns={0:'tarifas'})
)
tarifas_list

,unique,tarifas
0,0044e06c-d45a-4ff1-907c-1d9ac4848945,"[{'cci': '0601', 'descricao': 'CONSUMO EM KWH'..."
1,02e9ef72-4a15-453e-a164-688bc64c189b,"[{'cci': '0602', 'descricao': 'DEMANDA POTÊNCI..."
2,051499f5-81ef-47a2-979a-8178e9915b53,"[{'cci': '0601', 'descricao': 'CONSUMO EM KWH'..."
3,093812c1-ab23-409f-8ff2-9dfffddbb64f,"[{'cci': '0601', 'descricao': 'CONSUMO EM KWH'..."
4,0bc7b143-9a0b-4c7d-b4dc-d5f68fd33775,"[{'cci': '0601', 'descricao': 'CONSUMO EM KWH ..."
...,...,...
157,f7cc0a2d-6c38-4b6b-9e9a-b46022c8af23,"[{'cci': '0602', 'descricao': 'DEMANDA POTÊNCI..."
158,f8c5ef05-fa88-4fb6-b0c7-2fc64134a166,"[{'cci': '0601', 'descricao': 'CONSUMO EM KWH'..."
159,fe1ad27b-808d-43b7-af3c-7da2ff2b551d,"[{'cci': '0601', 'descricao': 'CONSUMO EM KWH'..."
160,fe9e3273-73ca-49cf-8ea0-3a31984b4935,"[{'cci': '0601', 'descricao': 'CONSUMO EM KWH ..."


Como os dados `unique` não estão ordenados, será necessário fazer um merge.

In [36]:
# Dataframe com todos dados normalizados
norm_df = pd.merge(
	left=df.drop(columns='tarifas'),
	right=tarifas_list,
	on='unique'
)
print(norm_df.dtypes)
norm_df.head()

_id                                  object
estruturaConsumo                     object
unique                               object
cliente                              object
classe                               object
ligacao                              object
referencia                   datetime64[ns]
data_vencimento              datetime64[ns]
data_emissao                 datetime64[ns]
data_apresentacao            datetime64[ns]
data_proxima_leitura         datetime64[ns]
cnpj                                 object
uc                                   object
refaturamento                          bool
createdAt               datetime64[ns, UTC]
updatedAt               datetime64[ns, UTC]
tarifas                              object
dtype: object


,_id,estruturaConsumo,unique,cliente,classe,ligacao,referencia,data_vencimento,data_emissao,data_apresentacao,data_proxima_leitura,cnpj,uc,refaturamento,createdAt,updatedAt,tarifas
0,62220ff138b5b03262a806d4,"{'un': 'KWH', 'posto': 'PONTA', 'medido': 2436...",b45dc7a8-2aa8-4693-a7c2-2260c4f0b642,SUPERMERCADO PAREDAO LTDA,COMERCIAL / OUTROS SERVIÇOS E OUTRAS ATIVIDADES,TRIFASICO,2022-02-01,2022-04-11,2022-02-26,2022-03-04,2022-03-29,28.847.082/0001-23,6/732146-6,False,2022-03-04 13:11:13.417000+00:00,2022-03-04 13:11:13.417000+00:00,"[{'cci': '0601', 'descricao': 'CONSUMO EM KWH'..."
1,62220ff138b5b03262a806da,"{'un': 'KWH', 'posto': 'PONTA', 'medido': 1005...",1bcac16e-289a-45c4-8509-f98ad5b66bcb,TELEFONICA BRASIL SA,COMERCIAL / SERVIÇOS DE COMUNICAÇÕES E TELECOM...,TRIFASICO,2022-02-01,2022-03-15,2022-02-26,2022-03-08,2022-03-29,02.558.157/0001-62,6/2669086-7,False,2022-03-04 13:11:13.565000+00:00,2022-03-04 13:11:13.565000+00:00,"[{'cci': '0601', 'descricao': 'CONSUMO EM KWH'..."
2,62220ff238b5b03262a806e0,"{'un': 'KWH', 'posto': 'PONTA', 'medido': 1000...",37399592-abd1-4de3-bb28-1b3753caf972,TELEFONICA BRASIL SA,COMERCIAL / SERVIÇOS DE COMUNICAÇÕES E TELECOM...,BIFASICO,2022-02-01,2022-03-28,2022-02-26,2022-03-11,2022-03-29,02.558.157/0001-62,6/2668744-2,False,2022-03-04 13:11:14.121000+00:00,2022-03-04 13:11:14.121000+00:00,"[{'cci': '0601', 'descricao': 'CONSUMO EM KWH'..."
3,62220ff338b5b03262a806ee,"{'un': 'KWH', 'posto': 'PONTA', 'medido': 1856...",c0185cf7-f793-4c23-a593-b1ec1d41219a,COMERCIAL IOMAR DE ALIMENTOS LTDA EPP,COMERCIAL / COMERCIAL,TRIFASICO,2022-02-01,2022-04-13,2022-03-02,2022-03-04,2022-03-31,01.436.106/0001-03,6/336632-5,False,2022-03-04 13:11:15.541000+00:00,2022-03-04 13:11:15.541000+00:00,"[{'cci': '0601', 'descricao': 'CONSUMO EM KWH ..."
4,62220ff538b5b03262a80702,"{'un': 'INJ', 'posto': 'PONTA', 'medido': 0.0,...",161ce259-5c70-4818-b2c5-2eee335e5067,CONSORCIO GERADOR FAZENDA MODELO,COMERCIAL / OUTROS SERVIÇOS E OUTRAS ATIVIDADES,TRIFASICO,2022-03-01,2022-03-16,2022-03-03,2022-03-09,2022-03-30,33.372.528/0001-40,6/2731936-7,False,2022-03-04 13:11:17.866000+00:00,2022-03-04 13:11:17.866000+00:00,"[{'cci': '0602', 'descricao': 'DEMANDA POTÊNCI..."


# 6. <span id='item6'>Análise dos dados

## Distribuição das medições por cliente

In [39]:
# Visualizando os registros únicos agrupados por cliente
df.groupby(by='cliente')[['classe', 'ligacao', 'cnpj', 'uc']].nunique()

,classe,ligacao,cnpj,uc
cliente,,,,
AGUAS CUIABA S A CONCESSIONARIA DE SERVICOS,2,1,1,1
AGUAS LEBRINHA LTDA,2,1,1,1
COMERCIAL IOMAR DE ALIMENTOS LTDA EPP,1,1,1,1
COMERCIO DE GENEROS ALIMENTICIOS V S LTDA - ME,1,1,1,1
CONSORCIO GERADOR FAZENDA MODELO,2,1,1,1
IGUACU SUPERMERCADOS LTDA,1,1,1,1
J C ANDRADE SOUZA ME,1,1,1,1
M ALVORADA LTDA EPP,1,1,1,1
MINERACAO COITE LTDA,4,2,1,8


Como o cliente 'TELEFONICA BRASIL SA' possuí mais dados em quantidade e variedade, vamos utilizá-lo como amostra para visualizar as distribuições do consumo e tarifas.

In [40]:
cliente = 'TELEFONICA BRASIL SA'
query_data = norm_df[norm_df['cliente']==cliente]
# Normalizando os valores das tarifas
query_data = query_data.explode('tarifas')
query_data = pd.concat([
	query_data.drop(columns='tarifas'),
	query_data['tarifas'].apply(pd.Series)
], axis=1)
# Normalizando o valores de estruturaConsumo
query_data = pd.concat([
	query_data.drop(columns='estruturaConsumo'),
	query_data['estruturaConsumo'].apply(pd.Series)
], axis=1)
query_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 345 entries, 1 to 161
Data columns (total 44 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   _id                   345 non-null    object             
 1   unique                345 non-null    object             
 2   cliente               345 non-null    object             
 3   classe                345 non-null    object             
 4   ligacao               345 non-null    object             
 5   referencia            345 non-null    datetime64[ns]     
 6   data_vencimento       345 non-null    datetime64[ns]     
 7   data_emissao          345 non-null    datetime64[ns]     
 8   data_apresentacao     345 non-null    datetime64[ns]     
 9   data_proxima_leitura  345 non-null    datetime64[ns]     
 10  cnpj                  345 non-null    object             
 11  uc                    345 non-null    object             
 12  refatura

In [41]:
query_data.head(2)

,_id,unique,cliente,classe,ligacao,referencia,data_vencimento,data_emissao,data_apresentacao,data_proxima_leitura,cnpj,uc,refaturamento,createdAt,updatedAt,cci,descricao,quantidade,semTributos,comTributos,valorTotal,baseIcms,aliqIcms,icms,basePisCofins,pis,cofins,_id.$oid,class_desc,tarifa_posto,un,posto,medido,faturado,_id.$oid,saldoPonta,saldoForaPonta,expiraForaPonta,expiraPonta,saldoAcumulado,expiraAcumulado,leituraAnterior,leituraAtual,unique
1,62220ff138b5b03262a806da,1bcac16e-289a-45c4-8509-f98ad5b66bcb,TELEFONICA BRASIL SA,COMERCIAL / SERVIÇOS DE COMUNICAÇÕES E TELECOM...,TRIFASICO,2022-02-01,2022-03-15,2022-02-26,2022-03-08,2022-03-29,02.558.157/0001-62,6/2669086-7,False,2022-03-04 13:11:13.565000+00:00,2022-03-04 13:11:13.565000+00:00,0601,CONSUMO EM KWH,1005.0,0.6835,0.85548,859.76,859.76,17.0,146.16,713.60,4.76,21.92,62220ff138b5b03262a806db,CONSUMO,NA,KWH,PONTA,1005.0,1005.0,62220ff138b5b03262a806df,0,0,0,0,0,0,2022-01-28,2022-02-25,1bcac16e-289a-45c4-8509-f98ad5b66bcb
1,62220ff138b5b03262a806da,1bcac16e-289a-45c4-8509-f98ad5b66bcb,TELEFONICA BRASIL SA,COMERCIAL / SERVIÇOS DE COMUNICAÇÕES E TELECOM...,TRIFASICO,2022-02-01,2022-03-15,2022-02-26,2022-03-08,2022-03-29,02.558.157/0001-62,6/2669086-7,False,2022-03-04 13:11:13.565000+00:00,2022-03-04 13:11:13.565000+00:00,0601,ENERGIA ATV INJETADA OUC 02/2022 MPT,529.0,0.6835,0.76897,-406.79,-183.36,17.0,-31.17,-375.62,-2.50,-11.54,62220ff138b5b03262a806dc,ENERGIA,NA,KWH,PONTA,1005.0,1005.0,62220ff138b5b03262a806df,0,0,0,0,0,0,2022-01-28,2022-02-25,1bcac16e-289a-45c4-8509-f98ad5b66bcb


## Explorando `estruturaConsumo`

In [42]:
consumo_df.groupby(['un', 'posto']).describe()

medido                                                   \
             count           mean            std     min        25%   
un   posto                                                            
DRE  FPONTA   20.0     220.079830     468.318089    0.00     0.4300   
     PONTA    12.0      36.732025      34.941622    0.40    15.3825   
ERE  FPONTA   23.0     548.226357     913.954384    0.00     0.2500   
     PONTA    13.0     190.204885     346.703770    0.00     1.2500   
INJ  FPONTA   32.0  520452.377188  685641.461979    0.00     0.0000   
     PONTA    32.0   53941.500938   65885.498443    0.00     0.0000   
KVA  PONTA     9.0    9288.777778    8270.222167  779.00   861.0000   
KW   FPONTA   49.0     587.093592    1154.279719    0.00     0.0000   
     PONTA    21.0    1099.668990    1563.296100    0.00    17.8900   
KWH  FPONTA   21.0    9071.275419   13809.584213    0.00     0.0000   
     PONTA   125.0    5606.853163   10809.803681    0.00  1293.0000   
ULTP FPONTA    5.0    1169.974000    1340.343548  179.19   196.3800   

                                                     faturado               \
                     50%           75%           max    count         mean   
un   posto                                                                   
DRE  FPONTA      35.0800      50.63775  1.357780e+03     20.0    16.983000   
     PONTA       33.2350      41.30500  1.259192e+02     12.0     0.316667   
ERE  FPONTA      68.0000     910.00000  3.550000e+03     23.0   316.443748   
     PONTA        7.9335     198.23000  1.141000e+03     13.0   190.204885   
INJ  FPONTA  283786.0000  631232.75000  2.308221e+06     32.0  4921.388400   
     PONTA    33045.5000   71877.25000  2.162952e+05     32.0   586.930925   
KVA  PONTA    13448.0000   15662.00000  2.041800e+04      9.0     0.000000   
KW   FPONTA       0.0000     135.52960  3.494040e+03     49.0  1122.180335   
     PONTA       39.4600    2681.95000  3.491920e+03     21.0     5.714286   
KWH  FPONTA       2.4000   17753.64000  5.076484e+04     21.0  9071.227800   
     PONTA     2021.0000    3213.00000  6.428800e+04    125.0  5607.469163   
ULTP FPONTA     198.7100    2592.00000  2.683590e+03      5.0    75.118000   

                                                                            \
                      std  min      25%        50%         75%         max   
un   posto                                                                   
DRE  FPONTA     44.712768  0.0     0.00     0.0000      1.2275    159.0700   
     PONTA       0.573781  0.0     0.00     0.0000      0.2950      1.3200   
ERE  FPONTA    618.714976  0.0     0.00     3.2000    128.9081   1775.9800   
     PONTA     346.703770  0.0     1.25     7.9335    198.2300   1141.0000   
INJ  FPONTA  10891.472713  0.0     0.00     0.0000    344.5000  50764.8388   
     PONTA    1382.345447  0.0     0.00     0.0000      6.9650   6804.1796   
KVA  PONTA       0.000000  0.0     0.00     0.0000      0.0000      0.0000   
KW   FPONTA   1110.001129  0.0    57.00  1000.0000   1000.0000   3500.0000   
     PONTA      14.342743  0.0     0.00     0.0000      0.0000     40.0000   
KWH  FPONTA  13809.617055  0.0     0.00     2.4000  17753.6400  50764.8388   
     PONTA   10809.486372  0.0  1293.00  2021.0000   3213.0000  64288.0000   
ULTP FPONTA    107.836356  0.0     0.00     0.0000    142.0000    233.5900   

            saldoPonta                                                    \
                 count           mean           std       min        25%   
un   posto                                                                 
DRE  FPONTA       17.0   45681.941176  52584.749378       0.0     621.00   
     PONTA         9.0   22851.111111  35547.676371       0.0       1.00   
ERE  FPONTA       20.0   56102.950000  54560.007251       0.0     621.75   
     PONTA        10.0   14131.300000  35476.492900       0.0       0.25   
INJ  FPONTA       29.0   95992.310345  81911.290578       0.0   13331.00

Criando uma *feature* `saldo_medido` = `faturado` - `medido`

In [43]:
consumo_df['saldo_medido'] = consumo_df['faturado'] - consumo_df['medido']
# Plotando a distribuição
px.box(
	data_frame=consumo_df, 
	x='posto', 
	y='saldo_medido', 
	color='un',
	title='Distribuição do saldo medido por grupo', 
	width=800, 
	color_discrete_sequence=CATEGORICAL_GROUPS[consumo_df['un'].nunique()]
).update_layout(**LAYOUT_SPECS)

A partir desta visualização podemos tirar conslusões de cada medição:  
- `KWH`: com excessão de poucos outliers, todas as medições possuem saldo zero.
- `KW`: observa-se que na categoria `PONTA` o saldo tem distribuição negativa, e na `FPONTA` o saldo é positivo. Como a demanda é contratada, isso significa que em geral os locais estão abaixo da demanda em períodos de FPONTA e acima dela nos períodos de PONTA.  
- `ERE`: encontra-se alguns outliers(n=4) medidos no período de FPONTA.  
- `DRE`: em ambos períodos o saldo é negativo, indicando que o valor medido é superior ao faturado.  
- `KVA`: esta medida está presente em poucas observações(n=9), seu valor é negativo pois esse dado é apenas medido, não é cobrado na fatura.  
- `INJ`: valores negativos podem significar uma grande quantidade de energia injetada proveniente de geração própria, por isso não é faturada.  
- `ULTP`: a ultrapassagem foi registrada em apenas em períodos de `FPONTA`.  

## Explorando `tarifas`

# 7. <span id='item7'>Apresentação em *Dashboards*

Visualizando dados por cliente

In [44]:
def select_client(name):
	cliente_query = norm_df[norm_df['cliente']==name]

	tarifas_cliente = pd.concat([
		cliente_query.drop(columns='tarifas'),
		cliente_query.explode('tarifas')['tarifas'].apply(pd.Series)
	], axis=1)

	consumo_cliente = pd.concat([
		cliente_query.drop(columns='estruturaConsumo'),
		cliente_query['estruturaConsumo'].apply(pd.Series)
	], axis=1)

	return tarifas_cliente, consumo_cliente

In [45]:
cliente = 'AGUAS LEBRINHA LTDA'
cliente_query = norm_df[norm_df['cliente']==cliente]

In [46]:
tarifas_cliente = pd.concat([
	cliente_query.drop(columns='tarifas'),
	cliente_query.explode('tarifas')['tarifas'].apply(pd.Series)
], axis=1)

px.bar(
	tarifas_cliente,
	x='referencia',
	y='quantidade',
	color='class_desc',
	#facet_row='posto',
	facet_col='uc',
	custom_data=['descricao'],
	barmode='group',
	title=f'{cliente}<br><sup>Histórico por unidade consumidora</sup>',
	color_discrete_sequence=CATEGORICAL_GROUPS[tarifas_cliente['class_desc'].nunique()],
	height=600, width=1000
).update_traces(
    hovertemplate="<br>".join([
        "Data: %{x}",
        "Valor medido: %{y}",
        "Descrição: %{customdata[0]}",
    ])
).update_layout(**LAYOUT_SPECS
).update_yaxes(matches=None, **LAYOUT_SPECS['yaxis']
).for_each_yaxis(lambda yaxis: yaxis.update(showticklabels=True))

In [47]:
consumo_cliente = pd.concat([
	cliente_query.drop(columns='estruturaConsumo'),
	cliente_query['estruturaConsumo'].apply(pd.Series)
], axis=1)

consumo_cliente['saldo'] = consumo_cliente['faturado'] - consumo_cliente['medido']

px.bar(
	consumo_cliente,
	x='referencia',
	y='medido',
	color='un',
	facet_col='posto',
	title=f'{cliente}<br><sup>Histórico das medições</sup>',
	color_discrete_sequence=CATEGORICAL_GROUPS[consumo_cliente['un'].nunique()],
	width=1000
).update_layout(**LAYOUT_SPECS
).update_xaxes(**LAYOUT_SPECS['xaxis'])

In [48]:
t1, c1 = select_client('TELEFONICA BRASIL SA')

In [49]:
px.box(
	t1,
	x='referencia',
	y='quantidade',
	color='class_desc',
	facet_col='posto',
	title=f'{cliente}<br><sup>Histórico das medições</sup>',
	color_discrete_sequence=CATEGORICAL_GROUPS[7],
	width=1000
).update_layout(**LAYOUT_SPECS
).update_xaxes(**LAYOUT_SPECS['xaxis'])

ValueError: Value of 'facet_col' is not the name of a column in 'data_frame'. Expected one of ['_id', 'estruturaConsumo', 'unique', 'cliente', 'classe', 'ligacao', 'referencia', 'data_vencimento', 'data_emissao', 'data_apresentacao', 'data_proxima_leitura', 'cnpj', 'uc', 'refaturamento', 'createdAt', 'updatedAt', 'cci', 'descricao', 'quantidade', 'semTributos', 'comTributos', 'valorTotal', 'baseIcms', 'aliqIcms', 'icms', 'basePisCofins', 'pis', 'cofins', '_id.$oid', 'class_desc', 'tarifa_posto'] but received: posto

In [ ]:
px.box(
	c1,
	x='referencia',
	y='medido',
	color='un',
	facet_col='posto',
	title=f'{cliente}<br><sup>Histórico das medições</sup>',
	color_discrete_sequence=CATEGORICAL_GROUPS[c1['un'].nunique()],
	width=1000
).update_layout(**LAYOUT_SPECS
).update_xaxes(**LAYOUT_SPECS['xaxis'])

# 8. <span id='item8'>Possíveis aplicações de ML